# Objectif : Comparer les produits structurés, effectuer du clustering pour identifier les outliers

Nous allons faire un embedding des descriptions de produits structurés ainsi que des caractéristiques que nous avons scrappé pour déterminer un score de similarité. Ensuite, nous effectuerons des algorithmes de clustering sur ce score pour déterminer qui sont les outliers, sous-entendu les produits structurés complexes.

Pour cela, il faudra faire apparaitre dans le résumé le détail du fonctionnement du produit structuré afin de pouvoir vraiment comparer.

In [3]:
import os
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
import tiktoken
from openai import AzureOpenAI

In [7]:
df=pd.read_csv(os.path.join(os.getcwd(),'output/bdd_DIC.csv')) 

print(list(df.columns))

['code_ISIN', 'nom_du_produit', 'emetteur_du_produit', 'date_emission', 'date_remboursement', 'mention_complexite', 'montant_minimum_investissement', 'niveau_garantie', 'niveau_barriere_desactivante', 'niveau_risque', 'produit_sous_jacent', 'nature_sous_jacent', 'code_ISIN_sous_jacent', 'frais_ponctuels_entree', 'frais_ponctuels_sortie', 'frais_recurrents', 'frais_accessoires', 'performance_tension', 'performance_maximale', 'espérance_maximale_rendement', 'date_actualisation', 'frais_ponctuels_sortie_echeance', 'frais_ponctuels_sortie_anticipee', 'resume_mecanisme']


In [8]:

# columns = ['code_ISIN', 'nom_du_produit', 'emetteur_du_produit', 'date_emission', 'date_remboursement', 'mention_complexite', 'montant_minimum_investissement', 'niveau_garantie', 'niveau_barriere_desactivante', 'niveau_risque', 'produit_sous_jacent', 'nature_sous_jacent', 'code_ISIN_sous_jacent', 'frais_ponctuels_entree', 'frais_ponctuels_sortie', 'frais_recurrents', 'frais_accessoires', 'performance_tension', 'performance_maximale', 'espérance_maximale_rendement', 'date_actualisation', 'frais_ponctuels_sortie_echeance', 'frais_ponctuels_sortie_anticipee', 'resume_mecanisme']

pd.options.mode.chained_assignment = None 

# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    return s

df["resume_mecanisme"] = df["resume_mecanisme"].apply(lambda x: normalize_text(x))

tokenizer = tiktoken.get_encoding("cl100k_base")
df['n_tokens'] = df["resume_mecanisme"].apply(lambda x: len(tokenizer.encode(x)))


In [9]:
client = AzureOpenAI(
  api_key = "7e93421f46cd4680831023addcb0f42d",  
  api_version = "2024-02-15-preview",
  azure_endpoint = "https://francecentral-openai.openai.azure.com"
)

def generate_embeddings(text, model="ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

df['ada_v2'] = df["resume_mecanisme"].apply(lambda x : generate_embeddings (x, model = "ada-002")) 

In [10]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_embedding(text, model="ada-002"): # model = "deployment_name"
    return client.embeddings.create(input = [text], model=model).data[0].embedding

def search_docs(df, user_query, top_n=4, to_print=True):
    embedding = get_embedding(
        user_query,
        model="ada-002" # model should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )
    df["understandability"] = df.ada_v2.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("understandability", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res


res = search_docs(df, "This is a complex structured product.", top_n=4)

,code_ISIN,nom_du_produit,emetteur_du_produit,date_emission,date_remboursement,mention_complexite,montant_minimum_investissement,niveau_garantie,niveau_barriere_desactivante,niveau_risque,...,performance_tension,performance_maximale,espérance_maximale_rendement,date_actualisation,frais_ponctuels_sortie_echeance,frais_ponctuels_sortie_anticipee,resume_mecanisme,n_tokens,ada_v2,understandability
2,XS2442385998,Calliandra Mai 2028,BNP Paribas Issuance B.V.,2023-03-08,2028-06-01,Oui,1000 EUR,1,0,2,...,-13.01%,4.67%,4.67%,2024-02-28,0%,0.5%,Ce produit est un certificat offrant une prote...,82,"[-0.02036852203309536, -0.028857562690973282, ...",0.813934
0,FR001400A2R3,Autocall Dégressif sur LFDE,SG Issuer,2022-06-06,2032-05-06,Oui,1000.0,0,50,4,...,-79.62%,8.20%,8.20%,2024-02-28,0.00%,0.00%,Ce produit est un titre de créance non assorti...,110,"[-0.020077845081686974, -0.032778553664684296,...",0.812183
1,XS2372852637,Nemesia Septembre 2030,BNP Paribas Issuance B.V.,2022-09-01,2030-09-03,Oui,Non spécifié,0,50,5,...,-18.64%,4.56%,Non spécifié,2024-02-28,Non spécifié,Non spécifié,• Certificat indexé sur la performance de l'in...,203,"[-0.017707742750644684, -0.012890094891190529,...",0.772192
